In [ ]:
# Processing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re, random, time
from datasets import Dataset
from collections import Counter
from statistics import mean

import os
os.chdir('/content')
from google.colab import drive

  Using cached bertopic-0.16.2-py2.py3-none-any.whl (158 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.10
    Uninstalling Cython-3.0.10:
      Successfully uninstalled Cython-3.0.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("ExtractedTweets.csv", sep = ",")
df.head()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...
4,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...


#### Applying the same pre-processing

In [ ]:
# Removing the links
def remove_tokens(text):
    text = text.str.replace(r'https://\S+', '', regex = True)  # Remove URLs
    text = text.str.replace(r'@\S+', '', regex = True)         # Remove mentions
    text = text.str.replace(r'https…', '', regex = True)       # Remove https...
    text = text.str.replace(r'RT', '', regex = True)           # Remove RT
    return text

# Apply the function to the 'text_column'
df['Tweet'] = remove_tokens(df['Tweet'])

In [ ]:
num_tweets = len(df)
representatives = df.Handle.unique()
num_representatives = len(representatives)
print(f'Example tweet: {df["Tweet"][15]}')
print(f"There are {num_representatives} representatives and {num_tweets} tweets in total.")
print(f"Tweets per represeentative: {round(num_tweets/num_representatives, 2)} compared to expected 200")

# Dict with number of tweets per representative
tweets_per_rep: dict = {}
tweets_per_rep = dict(Counter(df['Handle']))
# Leaving only the representatives with exactly 200 tweets
reps_with_200 = [handle for handle, count in tweets_per_rep.items() if count == 200]
print(f"{len(reps_with_200)} representatives left out of {num_representatives}.")
# Filter the original DataFrame to only include reps with 200 tweets
filtered_df = df[df['Handle'].isin(reps_with_200)]
# Group tweets by user
grouped_tweets = filtered_df.groupby('Handle')['Tweet'].apply(lambda x: [list(x)]).reset_index()
assert len(grouped_tweets) == len(reps_with_200), 'Number of unique handles does not match'
print(f"{int(len(filtered_df)/num_tweets * 100)}% of tweets retained.")

Example tweet: Meet 12 incredible young leaders appointed to US Military Academies from our District!  
There are 433 representatives and 86460 tweets in total.
Tweets per represeentative: 199.68 compared to expected 200
416 representatives left out of 433.
96% of tweets retained.


In [ ]:
grouped_tweets.head()

,Handle,Tweet
0,AGBecerra,[[#Breaking: Court just granted our motion to ...
1,AlanGrayson,"[[ 📻TUNE IN📻\n\nMy hubby, and I will join o..."
2,AnthonyBrownMD4,[[ I've had the privilege of serving w some of...
3,AustinScottGA08,"[[This week, we have the opportunity to bring ..."
4,BennieGThompson,[[ H.R. 4895 by passed the House yesterday (...


In [ ]:
random_column: list  = []
# Select 25 random sentences
for index, row in grouped_tweets.iterrows():
    select_sent: list = random.sample(row['Tweet'][0], 25)
    random_column.append(select_sent)

grouped_tweets['Random_tweets'] = random_column

In [ ]:
assert len(grouped_tweets['Random_tweets']) == len(grouped_tweets['Tweet'])
assert len(grouped_tweets['Random_tweets'][0]) == 25

In [ ]:
grouped_tweets.to_csv('Dataset_random_selection.csv', index=True)